Getting the imports I need


In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch import nn
!pip install d2l
%pip install matplotlib-inline
from d2l import torch as d2l
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

Unzipping my files so I can get the images in the code


In [ ]:
!unzip /content/Training\ and\ Validation\ Data_2023_T1_COMP3010.zip

In [4]:
#the compose function applies transformations to each image in order
train_transforms = transforms.Compose([
    #randomly rotates image by 30 degrees(for data augmentation purposes)
    transforms.RandomRotation(30),
    #crops image to a square size of 224*224
    transforms.RandomResizedCrop(224),
    #flips image horizontally again for augmentation purposes
    transforms.RandomHorizontalFlip(),
    #converts image to pytorch tensor
    transforms.ToTensor(),
    #normalising pixel of image
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
#doing the same to validation and testing data too
val_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
#getting the data from the path and subdirectories
data_dir = '/content/Training and Validation Data_2023_T1_COMP3010 (2)/Training and Validation Data_2023_T1_COMP3010/Data/coral image classification'
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
val_data = datasets.ImageFolder(data_dir + '/val', transform=val_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)




In [8]:
#number of samples processed in each batch
batch_size = 16
#shuffle is for shuffling data before creating batches
#the train_data,val_data and test_data are datasets
#data loaders iterate over these datasets in batch sizes
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:


#loading pre-trained resnet model from the torch library
mod = torchvision.models.resnet50(pretrained = True)
#this one is not pre-trained has randomised weights
rnd = torchvision.models.resnet50()
#replaces last fully connected layer with new layer that has same input features as old but new output features,new output
#features equal to number of classes in training data


print(mod.fc.in_features)
print(len(train_data.classes))
rnd.fc = nn.Linear(rnd.fc.in_features, len(train_data.classes))
#checking if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
torch.cuda.is_available()

In [11]:
#defining a class that takes in from the nn.module and calling it CNN feature extractor
class CNNFeatureExtractor(nn.Module):
  #takes in an argument mod which in my case is a pretrained model resnet50
    def __init__(self, mod):
        super().__init__()
        #the nn.seq object is a container for the layers and uses list to get
        #all the pretrained layers and excludes out the last layer since thats what we need
        self.features = nn.Sequential(*list(mod.children())[:-1])
        #x is an input tensor
    def forward(self, x):
      #input tensor passed through convulutional layer
        x = self.features(x)
        #reshapes based on batch size and number ofc features
        x = x.view(x.size(0), -1)
        #returns the tensor
        return x


In [12]:
mod.to(device)
mod.eval()
#creating to empty lists with these names
features_train = []
labels_train = []
#just no gradients being computed
with torch.no_grad():
    #iterating through the trainloader I made initially
    for images, labels in trainloader:
        #using cuda
        images = images.to(device)
        labels = labels.to(device)
        #passing the images  through mod so i can get features
        features = mod(images)
        #converting features from tensor to numpy arrays
        features_train.extend(features.cpu().numpy().astype(np.float32)) # cast to np.float32
        labels_train.extend(labels.cpu().numpy())
#doing the same as above for validation and testing
features_val = []
labels_val = []
with torch.no_grad():
    for images, labels in valloader:
        images = images.to(device)
        labels = labels.to(device)
        features = mod(images)
        features_val.extend(features.cpu().numpy().astype(np.float32)) # cast to np.float32
        labels_val.extend(labels.cpu().numpy())

features_test = []
labels_test = []
with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        features = mod(images)
        features_test.extend(features.cpu().numpy().astype(np.float32)) # cast to np.float32
        labels_test.extend(labels.cpu().numpy())



In [ ]:
torch.cuda.is_available()

True

In [13]:
# training a simple model using extracted CNN features
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(features_train, labels_train)



KNeighborsClassifier()

In [18]:
# evaluating the performance on validation data
train_accuracy = knn.score(features_train, labels_train)
print('Training accuracy:', train_accuracy)

Training accuracy: 0.9418084153983886


In [19]:
# evaluating the performance on validation data
val_accuracy = knn.score(features_val, labels_val)
print('Validation accuracy:', val_accuracy)

Validation accuracy: 0.9275


In [20]:
# evaluating the performance on testing data
test_accuracy = knn.score(features_test, labels_test)
print('Testing accuracy:', test_accuracy)



Testing accuracy: 0.92
